# G₂ Universality Test v6

## Testing λ₁ × H* = dim(G₂) = 14 across different topologies

**Conjecture**: For any compact G₂ manifold with Betti numbers (b₂, b₃):
```
λ₁ × H* = 14 = dim(G₂)
where H* = b₂ + b₃ + 1
```

**TCS Ratio Formula**:
```
ratio*(H*) = H* / (6 × dim(G₂)) = H*/84
```

**Test Plan**:
1. Test known G₂ manifolds: Joyce, Kovalev, CHNP
2. Test synthetic (b₂, b₃) splits with same H*
3. Verify λ₁ depends only on H*, not on individual (b₂, b₃)
4. Confirm ratio* = H*/84 is the universal optimal ratio

---
*GIFT Framework - Universality Validation*

In [ ]:
# Cell 1: Imports and Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict
import json
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("GIFT Universality Test v6")
print("="*50)

In [ ]:
# Cell 2: G₂ Manifold Configuration
@dataclass
class G2Manifold:
    """Configuration for a G₂ manifold."""
    name: str
    b2: int           # Second Betti number
    b3: int           # Third Betti number
    source: str       # Construction type
    
    @property
    def H_star(self) -> int:
        return self.b2 + self.b3 + 1
    
    @property
    def predicted_ratio(self) -> float:
        """Predicted optimal TCS ratio = H*/84."""
        return self.H_star / 84
    
    @property
    def predicted_lambda1(self) -> float:
        """Predicted λ₁ = 14/H*."""
        return 14 / self.H_star
    
    @property
    def det_g(self) -> float:
        """Metric determinant = 65/32 (universal for G₂)."""
        return 65/32

# Define test manifolds
MANIFOLDS = [
    # GIFT K₇ (baseline)
    G2Manifold("K7_GIFT", 21, 77, "TCS-GIFT"),
    
    # Joyce orbifolds
    G2Manifold("Joyce_J1", 12, 43, "Joyce"),
    G2Manifold("Joyce_J2", 2, 10, "Joyce"),
    G2Manifold("Joyce_min", 0, 4, "Joyce"),
    G2Manifold("Joyce_large", 0, 103, "Joyce"),
    
    # Kovalev TCS
    G2Manifold("Kovalev_K1", 0, 71, "Kovalev"),
    G2Manifold("Kovalev_K2", 0, 155, "Kovalev"),
    
    # CHNP
    G2Manifold("CHNP_min", 0, 55, "CHNP"),
    G2Manifold("CHNP_max", 0, 239, "CHNP"),
    
    # Synthetic H*=99 splits (test independence from b₂/b₃ split)
    G2Manifold("Synth_99_a", 14, 84, "Synthetic"),
    G2Manifold("Synth_99_b", 35, 63, "Synthetic"),
    G2Manifold("Synth_99_c", 0, 98, "Synthetic"),
    G2Manifold("Synth_99_d", 49, 49, "Synthetic"),
]

print(f"Loaded {len(MANIFOLDS)} manifold configurations")
print("\n{:<15} {:>5} {:>5} {:>5} {:>8} {:>8}".format(
    "Name", "b₂", "b₃", "H*", "ratio*", "λ₁_pred"))
print("-"*55)
for M in MANIFOLDS:
    print("{:<15} {:>5} {:>5} {:>5} {:>8.4f} {:>8.4f}".format(
        M.name, M.b2, M.b3, M.H_star, M.predicted_ratio, M.predicted_lambda1))

In [ ]:
# Cell 3: Quaternion Operations (from v5)
class Quaternion:
    """Quaternion operations for S³ ≅ SU(2)."""
    
    @staticmethod
    def normalize(q: np.ndarray) -> np.ndarray:
        if q.ndim == 1:
            return q / np.linalg.norm(q)
        return q / np.linalg.norm(q, axis=1, keepdims=True)
    
    @staticmethod
    def random_unit(n: int, seed: Optional[int] = None) -> np.ndarray:
        if seed is not None:
            np.random.seed(seed)
        q = np.random.randn(n, 4)
        return Quaternion.normalize(q)
    
    @staticmethod
    def geodesic_distance(q1: np.ndarray, q2: np.ndarray) -> np.ndarray:
        """Geodesic distance on S³: d(q₁,q₂) = 2·arccos(|⟨q₁,q₂⟩|)."""
        if q1.ndim == 1 and q2.ndim == 1:
            dot = np.abs(np.dot(q1, q2))
        else:
            dot = np.abs(np.sum(q1 * q2, axis=-1))
        dot = np.clip(dot, 0.0, 1.0)
        return 2.0 * np.arccos(dot)

print("Quaternion class loaded")

In [ ]:
# Cell 4: Universal TCS Sampler
class UniversalTCSSampler:
    """
    Sample on S¹ × S³ × S³ with H*-dependent optimal ratio.
    
    Key insight: ratio*(H*) = H*/84 varies with topology.
    """
    
    def __init__(self, n_points: int = 5000, seed: int = 42):
        self.n_points = n_points
        self.seed = seed
        self.theta = None
        self.q1 = None
        self.q2 = None
        
    def sample(self):
        """Generate sample points."""
        np.random.seed(self.seed)
        self.theta = np.random.uniform(0, 2*np.pi, self.n_points)
        self.q1 = Quaternion.random_unit(self.n_points)
        self.q2 = Quaternion.random_unit(self.n_points)
    
    def compute_distance_matrix(self, H_star: int, ratio: float) -> np.ndarray:
        """
        Compute distance matrix with H*-normalized metric.
        
        Metric: ds² = α dθ² + ds₁² + ratio² ds₂²
        with α chosen so det(g) = 65/32.
        """
        n = self.n_points
        
        # Metric normalization: α × 1³ × ratio³ = 65/32
        alpha = (65/32) / (ratio**3)
        
        # S¹ distances (circular)
        theta_diff = np.abs(self.theta[:, None] - self.theta[None, :])
        d_S1 = np.minimum(theta_diff, 2*np.pi - theta_diff)
        
        # S³ geodesic distances
        d_S3_1 = np.zeros((n, n))
        d_S3_2 = np.zeros((n, n))
        for i in range(n):
            d_S3_1[i, :] = Quaternion.geodesic_distance(self.q1[i], self.q1)
            d_S3_2[i, :] = Quaternion.geodesic_distance(self.q2[i], self.q2)
        
        # TCS metric distance
        D = np.sqrt(alpha * d_S1**2 + d_S3_1**2 + (ratio**2) * d_S3_2**2)
        return D

print("UniversalTCSSampler loaded")

In [ ]:
# Cell 5: Graph Laplacian Spectrum
class GraphLaplacianSpectrum:
    """Compute spectrum of graph Laplacian."""
    
    def __init__(self, k_neighbors: int = 30):
        self.k_neighbors = k_neighbors
        
    def compute(self, D: np.ndarray) -> Tuple[np.ndarray, float]:
        """
        Compute first non-zero eigenvalue.
        
        Returns: (eigenvalues, lambda_1)
        """
        n = D.shape[0]
        
        # Adaptive bandwidth
        knn_dists = np.sort(D, axis=1)[:, 1:self.k_neighbors+1]
        sigma = np.median(knn_dists)
        
        # Gaussian kernel
        W = np.exp(-D**2 / (2 * sigma**2))
        np.fill_diagonal(W, 0)
        
        # k-NN sparsification
        for i in range(n):
            idx = np.argsort(W[i])[:-(self.k_neighbors+1)]
            W[i, idx] = 0
        W = (W + W.T) / 2
        
        # Normalized Laplacian
        d = np.sum(W, axis=1)
        d_inv_sqrt = np.where(d > 0, 1.0 / np.sqrt(d), 0)
        D_inv_sqrt = sp.diags(d_inv_sqrt)
        W_sparse = sp.csr_matrix(W)
        L = sp.eye(n) - D_inv_sqrt @ W_sparse @ D_inv_sqrt
        
        # Compute eigenvalues
        eigenvalues, _ = eigsh(L.tocsr(), k=10, which='SM')
        eigenvalues = np.sort(eigenvalues)
        
        return eigenvalues, eigenvalues[1]  # λ₁ is second eigenvalue

print("GraphLaplacianSpectrum loaded")

In [ ]:
# Cell 6: Universality Tester
class UniversalityTester:
    """
    Test the universality conjecture λ₁ × H* = 14.
    """
    
    def __init__(self, n_points: int = 3000, k_neighbors: int = 25, seed: int = 42):
        self.n_points = n_points
        self.k_neighbors = k_neighbors
        self.seed = seed
        
        self.sampler = UniversalTCSSampler(n_points, seed)
        self.spectrum = GraphLaplacianSpectrum(k_neighbors)
        self.results = []
    
    def test_manifold(self, M: G2Manifold, use_optimal_ratio: bool = True) -> Dict:
        """
        Test a single manifold configuration.
        
        If use_optimal_ratio=True, uses ratio* = H*/84.
        Otherwise, sweeps to find best ratio.
        """
        if self.sampler.theta is None:
            self.sampler.sample()
        
        if use_optimal_ratio:
            ratio = M.predicted_ratio
            D = self.sampler.compute_distance_matrix(M.H_star, ratio)
            eigenvalues, lambda1 = self.spectrum.compute(D)
            lambda1_H = lambda1 * M.H_star
            
            return {
                'name': M.name,
                'b2': M.b2,
                'b3': M.b3,
                'H_star': M.H_star,
                'ratio': ratio,
                'lambda1': lambda1,
                'lambda1_H': lambda1_H,
                'target': 14,
                'deviation_pct': abs(lambda1_H - 14) / 14 * 100
            }
        else:
            # Sweep ratios to find optimal
            best_result = None
            best_deviation = float('inf')
            
            for ratio in np.linspace(0.5, 2.5, 21):
                D = self.sampler.compute_distance_matrix(M.H_star, ratio)
                eigenvalues, lambda1 = self.spectrum.compute(D)
                lambda1_H = lambda1 * M.H_star
                deviation = abs(lambda1_H - 14)
                
                if deviation < best_deviation:
                    best_deviation = deviation
                    best_result = {
                        'name': M.name,
                        'b2': M.b2,
                        'b3': M.b3,
                        'H_star': M.H_star,
                        'ratio_optimal': ratio,
                        'ratio_predicted': M.predicted_ratio,
                        'lambda1': lambda1,
                        'lambda1_H': lambda1_H,
                        'target': 14,
                        'deviation_pct': best_deviation / 14 * 100
                    }
            
            return best_result
    
    def run_all(self, manifolds: List[G2Manifold], use_optimal_ratio: bool = True) -> List[Dict]:
        """Test all manifolds."""
        self.sampler.sample()
        self.results = []
        
        for M in tqdm(manifolds, desc="Testing manifolds"):
            result = self.test_manifold(M, use_optimal_ratio)
            self.results.append(result)
        
        return self.results

print("UniversalityTester loaded")

In [ ]:
# Cell 7: Run Test with Predicted Ratios
print("="*60)
print("TEST 1: Using predicted ratio* = H*/84")
print("="*60)

# Parameters
N_POINTS = 3000
K_NEIGHBORS = 25
SEED = 42

tester = UniversalityTester(N_POINTS, K_NEIGHBORS, SEED)
results_predicted = tester.run_all(MANIFOLDS, use_optimal_ratio=True)

print("\nResults with predicted ratio* = H*/84:")
print("-"*70)
print("{:<15} {:>5} {:>8} {:>10} {:>10} {:>10}".format(
    "Name", "H*", "ratio*", "λ₁×H*", "Target", "Dev %"))
print("-"*70)
for r in results_predicted:
    print("{:<15} {:>5} {:>8.4f} {:>10.4f} {:>10} {:>10.2f}%".format(
        r['name'], r['H_star'], r['ratio'], r['lambda1_H'], r['target'], r['deviation_pct']))

In [ ]:
# Cell 8: Run Test with Optimal Ratio Search
print("\n" + "="*60)
print("TEST 2: Finding optimal ratio by sweep")
print("="*60)

tester2 = UniversalityTester(N_POINTS, K_NEIGHBORS, SEED)
results_optimal = tester2.run_all(MANIFOLDS, use_optimal_ratio=False)

print("\nResults with optimal ratio search:")
print("-"*85)
print("{:<15} {:>5} {:>10} {:>10} {:>10} {:>10} {:>10}".format(
    "Name", "H*", "r_optimal", "r_pred", "λ₁×H*", "Target", "Dev %"))
print("-"*85)
for r in results_optimal:
    print("{:<15} {:>5} {:>10.4f} {:>10.4f} {:>10.4f} {:>10} {:>10.2f}%".format(
        r['name'], r['H_star'], r['ratio_optimal'], r['ratio_predicted'], 
        r['lambda1_H'], r['target'], r['deviation_pct']))

In [ ]:
# Cell 9: Test Independence from (b₂, b₃) Split
print("\n" + "="*60)
print("TEST 3: Independence from (b₂, b₃) split at H* = 99")
print("="*60)

# Filter H*=99 manifolds
h99_manifolds = [M for M in MANIFOLDS if M.H_star == 99]

print(f"\nTesting {len(h99_manifolds)} manifolds with H* = 99:")
print("-"*50)

h99_results = []
tester3 = UniversalityTester(N_POINTS, K_NEIGHBORS, SEED)
tester3.sampler.sample()

for M in h99_manifolds:
    result = tester3.test_manifold(M, use_optimal_ratio=True)
    h99_results.append(result)
    print(f"  {M.name:<15} (b₂={M.b2:>2}, b₃={M.b3:>2}): λ₁×H* = {result['lambda1_H']:.4f}")

# Compute spread
lambda_values = [r['lambda1_H'] for r in h99_results]
spread = (max(lambda_values) - min(lambda_values)) / np.mean(lambda_values) * 100

print("-"*50)
print(f"Mean λ₁×H*: {np.mean(lambda_values):.4f}")
print(f"Std λ₁×H*:  {np.std(lambda_values):.4f}")
print(f"Spread:     {spread:.2f}%")
print(f"\nConclusion: {'λ₁ depends ONLY on H*, not on (b₂,b₃) split' if spread < 5 else 'Spread too high - check methodology'}")

In [ ]:
# Cell 10: Universality Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Extract data
H_stars = [r['H_star'] for r in results_predicted]
lambda1_H_values = [r['lambda1_H'] for r in results_predicted]
ratios_pred = [r['ratio'] for r in results_predicted]
ratios_opt = [r['ratio_optimal'] for r in results_optimal]

# Plot 1: λ₁×H* vs H*
ax1 = axes[0, 0]
ax1.scatter(H_stars, lambda1_H_values, s=100, c='blue', alpha=0.7, edgecolors='black')
ax1.axhline(y=14, color='g', linestyle='--', linewidth=2, label='Target = 14')
ax1.set_xlabel('H* = b₂ + b₃ + 1', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Universality Test: λ₁ × H* vs H*', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 25])

# Plot 2: λ₁ vs 1/H* (should be linear)
ax2 = axes[0, 1]
inv_H_stars = [1/H for H in H_stars]
lambda1_values = [r['lambda1'] for r in results_predicted]
ax2.scatter(inv_H_stars, lambda1_values, s=100, c='red', alpha=0.7, edgecolors='black')

# Fit line λ₁ = c / H*
c_fit = np.mean([l * H for l, H in zip(lambda1_values, H_stars)])
x_line = np.linspace(min(inv_H_stars), max(inv_H_stars), 100)
ax2.plot(x_line, c_fit * x_line, 'g--', linewidth=2, label=f'Fit: λ₁ = {c_fit:.2f}/H*')

ax2.set_xlabel('1/H*', fontsize=12)
ax2.set_ylabel('λ₁', fontsize=12)
ax2.set_title('Linear Test: λ₁ vs 1/H*', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Optimal ratio vs Predicted ratio
ax3 = axes[1, 0]
ax3.scatter(ratios_pred, ratios_opt, s=100, c='purple', alpha=0.7, edgecolors='black')
ax3.plot([0, 3], [0, 3], 'k--', linewidth=1, label='y = x (perfect match)')
ax3.set_xlabel('Predicted ratio* = H*/84', fontsize=12)
ax3.set_ylabel('Optimal ratio (from sweep)', fontsize=12)
ax3.set_title('Ratio Formula Validation', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xlim([0, 3])
ax3.set_ylim([0, 3])

# Plot 4: H*=99 splits
ax4 = axes[1, 1]
h99_names = [r['name'] for r in h99_results]
h99_lambda = [r['lambda1_H'] for r in h99_results]
ax4.barh(h99_names, h99_lambda, color='teal', alpha=0.7)
ax4.axvline(x=14, color='g', linestyle='--', linewidth=2, label='Target = 14')
ax4.set_xlabel('λ₁ × H*', fontsize=12)
ax4.set_title('Independence Test: Different (b₂,b₃) with same H*=99', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('outputs/g2_universality_v6.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved: outputs/g2_universality_v6.png")

In [ ]:
# Cell 11: Statistical Summary
print("\n" + "="*60)
print("STATISTICAL SUMMARY")
print("="*60)

# Compute statistics
deviations = [r['deviation_pct'] for r in results_predicted]
mean_dev = np.mean(deviations)
std_dev = np.std(deviations)
max_dev = np.max(deviations)

print(f"\nUsing predicted ratio* = H*/84:")
print(f"  Mean deviation from 14: {mean_dev:.2f}%")
print(f"  Std deviation:          {std_dev:.2f}%")
print(f"  Max deviation:          {max_dev:.2f}%")

# Correlation
correlation = np.corrcoef(H_stars, lambda1_values)[0, 1]
print(f"\nCorrelation(H*, λ₁): {correlation:.4f}")

# Fit quality
predicted_lambda1 = [14/H for H in H_stars]
ss_res = sum((l - p)**2 for l, p in zip(lambda1_values, predicted_lambda1))
ss_tot = sum((l - np.mean(lambda1_values))**2 for l in lambda1_values)
r_squared = 1 - ss_res / ss_tot if ss_tot > 0 else 0

print(f"R² for λ₁ = 14/H*:   {r_squared:.4f}")

# Ratio formula validation
ratio_errors = [abs(r['ratio_optimal'] - r['ratio_predicted']) for r in results_optimal]
mean_ratio_error = np.mean(ratio_errors)

print(f"\nRatio formula validation (r* = H*/84):")
print(f"  Mean |r_optimal - r_predicted|: {mean_ratio_error:.4f}")

# Universality verdict
print("\n" + "="*60)
print("UNIVERSALITY VERDICT")
print("="*60)
if mean_dev < 10 and spread < 5:
    print("✓ SUPPORTED: λ₁ × H* ≈ 14 appears universal")
    print("✓ SUPPORTED: λ₁ depends only on H*, not on (b₂,b₃) split")
else:
    print("⚠ INCONCLUSIVE: Further investigation needed")

if mean_ratio_error < 0.3:
    print("✓ SUPPORTED: ratio* = H*/84 formula validated")
else:
    print("⚠ INCONCLUSIVE: Ratio formula needs refinement")

In [ ]:
# Cell 12: Save Results
import os
os.makedirs('outputs', exist_ok=True)

output = {
    'timestamp': datetime.now().isoformat(),
    'notebook': 'G2_Universality_v6',
    'parameters': {
        'n_points': N_POINTS,
        'k_neighbors': K_NEIGHBORS,
        'seed': SEED
    },
    'conjecture': 'λ₁ × H* = 14 = dim(G₂)',
    'ratio_formula': 'ratio* = H*/84',
    'results_with_predicted_ratio': results_predicted,
    'results_with_optimal_ratio': results_optimal,
    'h99_independence_test': h99_results,
    'statistics': {
        'mean_deviation_pct': mean_dev,
        'std_deviation_pct': std_dev,
        'max_deviation_pct': max_dev,
        'correlation_H_lambda': correlation,
        'r_squared': r_squared,
        'h99_spread_pct': spread
    },
    'conclusions': {
        'universality_supported': mean_dev < 10,
        'independence_supported': spread < 5,
        'ratio_formula_supported': mean_ratio_error < 0.3
    }
}

with open('outputs/g2_universality_v6_results.json', 'w') as f:
    json.dump(output, f, indent=2, default=str)

print("Results saved to: outputs/g2_universality_v6_results.json")
print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)

## Summary

### Universality Conjecture

```
λ₁ × H* = dim(G₂) = 14

for any compact G₂ manifold with H* = b₂ + b₃ + 1
```

### Key Tests

| Test | Description | Expected |
|------|-------------|----------|
| λ₁ × H* | Should equal 14 for all manifolds | 14 ± 10% |
| λ₁ vs 1/H* | Should be linear | R² > 0.9 |
| H*=99 splits | Same λ₁ regardless of (b₂,b₃) | Spread < 5% |
| ratio* | Should equal H*/84 | Error < 0.3 |

### Manifolds Tested

| Construction | Examples | H* Range |
|--------------|----------|----------|
| Joyce | J1, J2, min, large | 5 - 104 |
| Kovalev | K1, K2 | 72 - 156 |
| CHNP | min, max | 56 - 240 |
| GIFT K₇ | Baseline | 99 |
| Synthetic | H*=99 splits | 99 |

---
*GIFT Framework - Universality Validation*